# Utilisation of sleep diary with pyActigraphy

Sleep diary is usually given with an actimeter allowing participants to report their subjective evaluation of sleep episodes. Sleep diary is the easiest tool and the less invasive to get information about sleep episodes from participants. It allows comparison between data recorded by an actigraph and the real-life. 

In medical fields, sleep diary is commonly recommended in order to help doctors in the diagnosis and treatment of sleep-wake cycle disorders. 

In [ ]:
import pyActigraphy
import os

In [ ]:
from plotly.graph_objs import Figure, Layout, Scatter
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
# Retrieve path to example files
fpath = os.path.join(os.path.dirname(pyActigraphy.__file__),'tests/data/')

In [ ]:
# Read test file
raw = pyActigraphy.io.read_raw_awd(fpath+'example_01.AWD')

In [ ]:
# Check the start time of the actigraphy recording
raw.start_time

In [ ]:
# Check the duration of the recording
raw.duration()

## Read a sleep diary

Sleep diary class has been created to read a sleep diary file. Sleep diary file *must* be formatted as the following:

| SubjectID        | your_subject_id  |                  |
|------------------|------------------|------------------|
| Type             | Start            | End              |
| Night/Nap/NoWear | YYYY-MM-DD HH:MM | YYYY-MM-DD HH:MM |

As an illustration, let us first read the sleep diary of example 01:

In [ ]:
sleep_diary = raw.read_sleep_diary(fpath + 'example_01_sleepdiary.ods')

Subject's name in the sleep diary is easily verified:

In [ ]:
raw.sleep_diary.name

To access to the data contained in the sleep diary:

In [ ]:
raw.sleep_diary.diary

A `summary` function is available. It returns the count/mean/std/min/25%/50%/75%/max of the durations found for each state in the sleep diary:

In [ ]:
raw.sleep_diary.summary()

The mean and standard deviation for one state is accessible by `state_infos` function:

In [ ]:
raw.sleep_diary.state_infos('NIGHT')

Specific functions are implemented to directly estimate the mean and standard deviation for theses states:
* bed time
* nap time
* no wear

In [ ]:
raw.sleep_diary.total_bed_time()

In [ ]:
raw.sleep_diary.total_nap_time()

In [ ]:
raw.sleep_diary.total_nowear_time()

## Plot the data with sleep diary

First, create a layout for the plot and then plot data.

Shapes are created for each sleep episodes and are meant to be overlaid with the actimetry data. Different parameters (e.g. colours) enable to distinguish sleep during night-time and nap episode. 

All shapes are created from a template:

In [ ]:
raw.sleep_diary.shaded_area

Then plot actimetry data with the corresponding sleep diary, by adapting the layout:

In [ ]:
layout = Layout(
    title="Actigraphy data",
    xaxis=dict(title="Date time"),
    yaxis=dict(title="Counts/period"),
    shapes=raw.sleep_diary.shapes(),
    showlegend=False
)

In [ ]:
iplot(Figure(data=[Scatter(x=raw.data.index, y=raw.data)], layout=layout))

Features of shaded areas are easily accessible to modify them. Three types of modifications are feasible:
* Global modifications : change parameters for all shaded areas (by changing the shape template)
* One state modifications : change parameters specific for one state
* Local modifications : change parameters for the selected shaded area

In [ ]:
# Global modifications
raw.sleep_diary.shaded_area['opacity'] = 1

In [ ]:
layout.update(shapes=raw.sleep_diary.shapes());

In [ ]:
iplot(Figure(data=[Scatter(x=raw.data.index, y=raw.data)], layout=layout))

In [ ]:
# Access to colours for each state
raw.sleep_diary.state_colour

In [ ]:
# One state modification
raw.sleep_diary.state_colour['NIGHT'] = 'rgb(140,95,148)'

In [ ]:
# Check colour modification
raw.sleep_diary.state_colour

In [ ]:
layout.update(shapes=raw.sleep_diary.shapes());

layout = Layout(
    title="Actigraphy data",
    xaxis=dict(title="Date time"),
    yaxis=dict(title="Counts/period"),
    shapes=raw.sleep_diary.shapes(),
    showlegend=False
)

In [ ]:
iplot(Figure(data=[Scatter(x=raw.data.index, y=raw.data)], layout=layout))

In [ ]:
# Local modifications of the first shape
shapes = raw.sleep_diary.shapes()
shapes[0]['fillcolor'] = 'rgb(0,255,200)'

In [ ]:
layout.update(shapes=shapes);

In [ ]:
iplot(Figure(data=[Scatter(x=raw.data.index, y=raw.data)], layout=layout))

## Add custom states

For conveniency, four states (active, nap, night, nowear) are automatically implemented when a sleep diary is read. However, the *pyActigraphy* package allows users to customise colours and to add states.

In [ ]:
# Default implemented states 
raw.sleep_diary.state_index

To add a custom state, all indices and their colours have to be specified during the reading of the sleep diary:

In [ ]:
raw.read_sleep_diary(
    fpath + 'example_01_sleepdiary_extra_states.ods',
    state_index=dict(ACTIVE=2, NAP=1, NIGHT=0, NOWEAR=-1, AWAKE_IN_BED=3),
    state_colour=dict(
        NAP='#7bc043',
        NIGHT='#d3d3d3',
        NOWEAR='#ee4035',
        AWAKE_IN_BED='rgb(143, 19, 131)'
    )
)

In [ ]:
layout = Layout(
    title="Actigraphy data",
    xaxis=dict(title="Date time"), 
    yaxis=dict(title="Counts/period"),
    shapes=raw.sleep_diary.shapes(),
    showlegend=False)

In [ ]:
iplot(Figure(data=[Scatter(x=raw.data.index, y=raw.data)], layout=layout))

Mean and standard deviation for the added state are now also accessible:

In [ ]:
raw.sleep_diary.state_infos('AWAKE_IN_BED')